## KL divergence minimization

In this exercise, we will minimize KL divergence between the true distribution $p(x)$ and our model $q_\theta(x)$ using gradient descent. We will be utilizing [Tensorflow Probability (TFP)](https://www.tensorflow.org/probability) built on Tensorflow that makes it easy to combine probabilistic models and deep learning. If you haven't already, please go ahead and [install](https://www.tensorflow.org/probability/install) the library.


In [0]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
%matplotlib inline
import subprocess

FIGSIZE = (6, 6)

In [0]:
# Evaluate the probabilities given the samples
def prob_map(p, lim, N):
    x = np.linspace(lim[0], lim[1], N)
    y = np.linspace(lim[2], lim[3], N)
    xy = np.array(np.meshgrid(x, y)).reshape(2, N**2).T
    xy_prob = p.prob(xy.astype(np.float32)).eval()
    x, y = np.meshgrid(x, y)
    return x, y, xy_prob.reshape((N, N))

# Create mean and covariance params for bivariate normal distribution
def gen_gaussian_params(d=2):
    p_mean = np.random.standard_normal((d,)).astype(np.float32)
    p_cov = np.random.standard_normal((d, d)).astype(np.float32)
    p_cov[0, 1] = 0
    p_cov = np.dot(p_cov, p_cov.T)
    return p_mean, p_cov

# Plot helpers
def contour(x, y, xy_prob, lim, cmap=plt.cm.inferno, axis=None):
    if axis is None:
        plt.contour(x, y, xy_prob, np.linspace(0., 1.1*xy_prob.max(), 10),
            zorder=2, extent=lim, linewidths=4, colors='w', alpha=0.5)
        plt.contour(x, y, xy_prob, np.linspace(0., 1.1*xy_prob.max(), 10),
                zorder=3, extent=lim, linewidths=2, cmap=cmap)
    else:
        axis.contour(x, y, xy_prob, np.linspace(0., 1.1*xy_prob.max(), 10),
            zorder=2, extent=lim, linewidths=4, colors='w', alpha=0.5)
        axis.contour(x, y, xy_prob, np.linspace(0., 1.1*xy_prob.max(), 10),
                zorder=3, extent=lim, linewidths=2, cmap=cmap)
    
def scatter(sample, axis=None):
    if axis is None:
        plt.scatter(sample[:200, 0], sample[:200, 1], s=25, alpha=0.7, c='k')
    else:
        axis.scatter(sample[:200, 0], sample[:200, 1], s=25, alpha=0.7, c='k')

### Create a bivariate normal as the true distribution $p(x) = \mathcal{N}(\mu, \Sigma)$

Draw the probability contour plot for the true distribution p(x)

In [0]:
np.random.seed(555)
tf.reset_default_graph()
sess = tf.InteractiveSession()

p_mean, p_cov = gen_gaussian_params()
p_mean += 0.8
p = tfp.distributions.MultivariateNormalFullCovariance(p_mean, p_cov)

plt.figure(figsize=FIGSIZE)
N = 128
lim = [-2.5, 3.5, -2.5, 3.5]
x, y, xy_prob = prob_map(p, lim, N)
contour(x, y, xy_prob, lim, cmap=plt.cm.viridis)

plt.axis(lim)
plt.axis('off')
plt.show()

Draw samples from the true distribution $p(x)$ and visualize

In [0]:
n_sample = 1000
sample = p.sample(n_sample).eval()

plt.figure(figsize=FIGSIZE)
scatter(sample)
plt.axis(lim)
plt.axis('off')
plt.show()

## Part A. Forward KL minimization

### Create the model distribution 
Now we will create $q_\theta(x)$ and apply gradient descent to minimize the forward KL divergence
$$\theta^* = \arg\min_\theta D_{KL}(p(x) \parallel q_\theta(x))$$

In [0]:
# Define your model here!
q_mean = None
q_cov = None
q = tfp.distributions.MultivariateNormalFullCovariance(q_mean, q_cov)

# Define the loss here!
loss = None
train = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
sess.run(tf.global_variables_initializer())

Lets run it for 20 iterations and visualize $q_{\theta^{(t)}}(x)$ 

In [0]:
fig, axes = plt.subplots(5,4, figsize=[12, 15])

for i in range(5):
    for j in range(4):
        scatter(sample, axis=axes[i][j])
        x, y, xy_prob = prob_map(q, lim, N)
        contour(x, y, xy_prob, lim, axis=axes[i][j])
        axes[i][j].axis(lim)
        axes[i][j].axis('off')
        axes[i][j].set_title('iter: {}'.format(i*5 + j))
        sess.run(train)
        
plt.show()
plt.close(fig)

sess.close()

### Create a mixture of two bivariate normals as the true distribution
$p(x) = \pi_1 \mathcal{N}(\mu_1, \Sigma_1) + \pi_2 \mathcal{N}(\mu_2, \Sigma_2)$ where $\pi_1 + \pi_2 = 1$

In [0]:
def build_p(mix=0.8):
    return tfp.distributions.Mixture(
        cat=tfp.distributions.Categorical(probs=[mix, 1.-mix]),
        components=[
            tfp.distributions.MultivariateNormalFullCovariance(
                loc=[0.5, -0.7], covariance_matrix=[[1., 1.], [1., 2.]]),
            tfp.distributions.MultivariateNormalFullCovariance(
                loc=[-2.5, 2.5], covariance_matrix=[[0.5, 0.1], [0.1, 0.5]]),
        ])

np.random.seed(8)
tf.reset_default_graph()
sess = tf.InteractiveSession()

p = build_p()

Draw the probability contour plot for the true distribution p(x)

In [0]:
N = 128
lim = [-4, 4, -4, 4]
plt.figure(figsize=FIGSIZE)
x, y, xy_prob = prob_map(p, lim, N)
contour(x, y, xy_prob, lim, cmap=plt.cm.viridis)
plt.axis(lim)
plt.axis('off')
plt.show()

Draw samples from the true distribution $p(x)$ and visualize

In [0]:
n_sample = 1000
sample = p.sample(n_sample).eval()
plt.figure(figsize=FIGSIZE)
scatter(sample)
plt.axis(lim)
plt.axis('off')
plt.show()

### Create the model distribution 
Now we will create $q_\theta(x)$ and apply gradient descent to minimize the forward KL divergence
$$\theta^* = \arg\min_\theta D_{KL}(p(x) \parallel q_\theta(x))$$

In [0]:
# Define your model here!
q_mean = None
q_cov = None
q = tfp.distributions.MultivariateNormalFullCovariance(q_mean, q_cov)

# Define the loss here!
logp = None
train = tf.train.GradientDescentOptimizer(0.1).minimize(None)
sess.run(tf.global_variables_initializer())

Lets run it for 20 iterations and visualize $q_{\theta^{(t)}}(x)$ 

In [0]:
fig, axes = plt.subplots(5,4, figsize=[12, 15])

for i in range(5):
    for j in range(4):
        scatter(sample, axis=axes[i][j])
        x, y, xy_prob = prob_map(q, lim, N)
        contour(x, y, xy_prob, lim, axis=axes[i][j])
        axes[i][j].axis(lim)
        axes[i][j].axis('off')
        axes[i][j].set_title('iter: {}'.format(i*5 + j))
        sess.run(train)
        
plt.show()
plt.close(fig)

sess.close()

## Part B. Reverse KL minimization

In [0]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
p = build_p()

### Create the model distribution 
Now we will create $q_\theta(x)$ and apply gradient descent to minimize the reverse KL divergence
$$\theta^* = \arg\min_\theta D_{KL}(q_\theta(x) \parallel p(x))$$

In [0]:
# Define your model here!
q_mean = None
q_cov = None
q = tfp.distributions.MultivariateNormalFullCovariance(q_mean, q_cov)

# Define the loss here!
loss = None
train = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
sess.run(tf.global_variables_initializer())

Lets run it for 20 iterations and visualize $q_{\theta^{(t)}}(x)$ 

In [0]:
fig, axes = plt.subplots(5,4, figsize=[12, 15])

for i in range(5):
    for j in range(4):
        scatter(sample, axis=axes[i][j])
        x, y, xy_prob = prob_map(q, lim, N)
        contour(x, y, xy_prob, lim, axis=axes[i][j])
        axes[i][j].axis(lim)
        axes[i][j].axis('off')
        axes[i][j].set_title('iter: {}'.format(i*5 + j))
        sess.run(train)
        
plt.show()
plt.close(fig)

sess.close()